In [4]:
import pandas as pd
import uuid
from math import sin, cos, sqrt, atan2, radians
import dask.dataframe as dd



In [5]:
f_earthquakes_raw = pd.read_csv("../data/consolidated_data.csv") 
f_gdp_raw = pd.read_csv("../data/02.fred_gdp_usa.csv") 


In [9]:
f_earthquakes_raw.columns

Index(['Unnamed: 0', 'time', 'latitude', 'longitude', 'depth', 'mag',
       'magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place',
       'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

In [5]:
def get_year(time):
    time = str(time)
    year = time[0:4]
    return int(year)

In [11]:
f_earthquakes_raw = pd.read_csv("../data/consolidated_data.csv") 

f_earthquakes_raw['year'] = f_earthquakes_raw['time'].apply(get_year)

f_earthquakes_raw['id_earthquake']=f_earthquakes_raw.apply(lambda my_data: str(my_data['time'])+str(my_data['latitude'])+str(my_data['longitude']), axis = 1)
f_earthquakes_raw['id_earthquake']=f_earthquakes_raw.apply(lambda my_data: str(uuid.uuid5(uuid.NAMESPACE_DNS, my_data['id_earthquake'])), axis = 1)

f_earthquakes_raw['key_earthquake_gdp']=f_earthquakes_raw.apply(lambda my_data: str(my_data['year'])+str(my_data['latitude'])+str(my_data['longitude']), axis = 1)
f_earthquakes_raw['key_earthquake_gdp']=f_earthquakes_raw.apply(lambda my_data: str(uuid.uuid5(uuid.NAMESPACE_DNS, my_data['key_earthquake_gdp'])), axis = 1)

f_earthquakes_raw = f_earthquakes_raw.drop(columns=['Unnamed: 0'])

column_order = ['id_earthquake', 'key_earthquake_gdp', 'time', 'year', 'latitude', 'longitude',
 'depth', 'mag', 'magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id', 'place', 
 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 
 'locationSource', 'magSource', 'updated']
f_earthquakes_raw = f_earthquakes_raw.reindex(columns=column_order)

columns_renaming = {"updated":"updated_date"}

f_earthquakes_raw.rename(columns=columns_renaming ,inplace=True)

f_earthquakes_raw.to_parquet('../data/f_earthquakes.parquet', engine='fastparquet')



In [17]:
filter_cond = (f_earthquakes_raw["magType"] > 'ml') | (f_earthquakes_raw["magType"] > 'Ml') | (f_earthquakes_raw["magType"] > 'md') | (f_earthquakes_raw["magType"] > 'Md')

f_earthquakes_raw_after1995 = f_earthquakes_raw[(f_earthquakes_raw["time"] > '1995-01-01T00:00:00.000Z') & filter_cond]
f_earthquakes_raw_before1995 = f_earthquakes_raw[(f_earthquakes_raw["time"] < '1995-01-01T00:00:00.000Z') & filter_cond]

In [18]:
f_earthquakes_raw_after1995.min()

/var/folders/2_/bnd0fncj143gc6x61sb7k8mc0000gn/T/ipykernel_6466/2393015739.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  f_earthquakes_raw_after1995.min()


id_earthquake         000002b0-e3db-580a-bcde-f5aa08d7be1c
key_earthquake_gdp    000000b6-ec85-513a-b7df-7ac8dda33947
time                              1995-01-01T00:02:07.250Z
year                                                  1995
latitude                                           -84.422
longitude                                         -179.999
depth                                                -10.0
mag                                                  -9.99
magType                                                 Ml
nst                                                    0.0
gap                                                    0.0
dmin                                                   0.0
rms                                                   -1.0
net                                                     ak
id                                              ak-0000001
type                                            Rock Slide
horizontalError                                        0

In [41]:

f_earthquakes_raw.rename(columns=columns_renaming ,inplace=True)

In [44]:
f_earthquakes = pd.read_parquet('../data/extraction_layer/f_earthquakes.parquet', engine='fastparquet')


In [45]:
f_earthquakes.sample(5)

,id_earthquake,key_earthquake_gdp,time,year,latitude,longitude,depth,mag,magType,nst,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,updated_date
520017,6bbc964a-27ae-54c4-bd2b-07355a640454,1eecc87c-87cd-564a-9375-14a840791bbc,1989-01-11T15:15:16.030Z,1989,25.718000,95.978000,33.000,NaN,None,NaN,...,Myanmar,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us,2014-11-07T00:43:25.648Z
261196,aaf8d2ab-1a16-51a5-8113-fab779f8ee0f,9d3c4b6c-02d0-5f80-a70b-d21232d080c8,1984-02-07T22:21:28.790Z,1984,37.637833,-119.054167,0.863,1.71,md,24.0,...,Central California,earthquake,0.44,0.52,0.31,19.0,reviewed,nc,nc,2016-12-01T22:18:49.130Z
2968468,d2110c7b-f644-5b96-b567-1bc7f285d9e7,77990432-ce36-5567-bfb1-2ca92db4de59,2016-11-04T13:16:39.610Z,2016,38.753666,-122.723999,1.550,1.03,md,7.0,...,Northern California,earthquake,0.53,1.09,0.26,3.0,automatic,nc,nc,2017-02-11T06:01:37.382Z
14253,667e2103-17de-5273-80a3-eecfec46a4ce,210ca238-cd1a-59a9-8dca-d4782f6e43b2,1974-04-09T16:52:35.700Z,1974,62.930000,-149.594000,129.000,NaN,None,NaN,...,Central Alaska,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us,2014-11-06T23:21:23.660Z
2337438,9014c671-8865-5f85-a2df-40acd9197be3,d0780733-aafe-56e6-9931-d7b66176fe6f,2011-06-19T22:05:39.247Z,2011,59.739800,-153.258000,113.900,1.60,ml,NaN,...,Southern Alaska,earthquake,NaN,0.70,NaN,NaN,reviewed,ak,ak,2018-07-06T23:31:05.105Z


In [38]:
f_earthquakes_raw.sample(5)

,id_earthquake,key_earthquake_gdp,time,year,latitude,longitude,depth,mag,magType,nst,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,updated
1662251,40f46933-a896-54d9-8ca7-397cabd70388,53ae580e-a89d-5770-a8b1-45bd9228038b,2004-12-16T17:15:03.500Z,2004,38.821833,-122.810667,2.525,1.21,md,18.0,...,Northern California,earthquake,0.34,0.44,0.15,8.0,reviewed,nc,nc,2017-01-09T23:06:43.757Z
793024,9673eec1-3a8b-5983-87b3-f9196b7dc4f0,25936f9d-ff93-5ed2-a23f-d7162cb1065a,1993-06-24T08:05:28.380Z,1993,15.784000,147.700000,42.400,4.80,mb,NaN,...,Mariana Islands region,earthquake,NaN,NaN,NaN,13.0,reviewed,us,us,2014-11-07T00:53:18.949Z
343556,f9ffb8b1-a1c6-510d-868c-f67bb78ebde2,d1f37e76-0bb1-5397-a760-ab3902cfa52a,1985-07-02T02:07:50.050Z,1985,33.475000,-116.470000,8.297,NaN,NaN,0.0,...,"21km ESE of Anza, CA",earthquake,NaN,1.12,NaN,NaN,reviewed,ci,ci,2016-02-04T01:55:11.820Z
2074972,97075aac-f977-5f68-8454-f9c838eb65b7,1444e0c5-74ef-5390-894f-6d1fc51891a6,2008-11-06T05:39:23.850Z,2008,19.662000,-66.185000,70.300,3.20,md,6.0,...,Puerto Rico region,earthquake,NaN,NaN,NaN,NaN,reviewed,rspr,rspr,2014-11-07T01:37:41.394Z
894431,1e2688c6-3faa-558d-878b-68b605c54266,b5ac837c-5d13-54f7-9b0b-cd5920cb942e,1994-11-21T11:55:30.450Z,1994,42.887000,145.646000,51.100,5.10,mb,NaN,...,"Hokkaido, Japan region",earthquake,NaN,5.00,NaN,76.0,reviewed,us,us,2014-11-07T00:56:49.712Z


In [28]:
f_gdp_raw['key_gdp_earthquake']=f_gdp_raw.apply(lambda my_data: str(my_data['year'])+str(my_data['latitude'])+str(my_data['longitude']), axis = 1)
f_gdp_raw['key_gdp_earthquake']=f_gdp_raw.apply(lambda my_data: str(uuid.uuid5(uuid.NAMESPACE_DNS, my_data['key_gdp_earthquake'])), axis = 1)

f_gdp_raw['key_gdp_earthquake']=f_gdp_raw['id_gdp']


In [30]:
f_gdp_raw.sample(10)

,Unnamed: 0,year,gdp,state_code,state_name,longitude,latitude,id_gdp,key_gdp_earthquake
821,21,2018,170352.9,NV,Nevada,-116.853723,39.515882,a8c0be45-c467-555c-b55b-539496da7607,a8c0be45-c467-555c-b55b-539496da7607
1104,4,2001,285635.9,VA,Virginia,-78.492772,37.123224,76c13a75-7303-59f7-ae3b-d95154874848,76c13a75-7303-59f7-ae3b-d95154874848
625,0,1997,19290.4,MT,Montana,-109.638757,47.375267,7a24626a-f57f-50b3-a9ea-6313cc5abd85,7a24626a-f57f-50b3-a9ea-6313cc5abd85
885,10,2007,145752.1,OK,Oklahoma,-97.268406,34.955082,3d390ea4-31dd-506f-ab02-db56decc395a,3d390ea4-31dd-506f-ab02-db56decc395a
900,0,1997,100914.8,OR,Oregon,-120.737257,43.979280,e6162366-d481-5c42-aec3-04297fde4e62,e6162366-d481-5c42-aec3-04297fde4e62
673,23,2020,589829.0,NC,North Carolina,-79.039292,35.672964,b00423d3-dad7-5da3-98d8-8a344ed3557e,b00423d3-dad7-5da3-98d8-8a344ed3557e
651,1,1998,246890.4,NC,North Carolina,-79.039292,35.672964,8df7b222-d0ea-5167-b3d5-2c76c2a4cebb,8df7b222-d0ea-5167-b3d5-2c76c2a4cebb
480,5,2002,217837.1,MD,Maryland,-76.938207,39.516223,b43417f8-097f-54fb-b4a4-9073af41eda0,b43417f8-097f-54fb-b4a4-9073af41eda0
1101,1,1998,231384.0,VA,Virginia,-78.492772,37.123224,531156ac-f3af-5bd7-bab4-a7773bb73948,531156ac-f3af-5bd7-bab4-a7773bb73948
874,24,2021,736449.8,OH,Ohio,-82.688140,40.225357,2f0e1f54-8149-5c30-af23-004cc247b03e,2f0e1f54-8149-5c30-af23-004cc247b03e


In [13]:


def calc_distance_math(lon_cent,lat_cent,lon_obj,lat_obj):
    '''
    '''
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat_cent)
    lon1 = radians(lon_cent)
    lat2 = radians(lat_obj)
    lon2 = radians(lon_obj)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

def generate_final_data(df_eq,df_gdp):
    
    '''Generates the final data to analyse with the earthquake and gdp data merged in same dataframe'''
    
    #importing csvs for earthquakes and gdp data
    df_eq = df_eq.drop(columns=['Unnamed: 0'])
    df_gdp = df_gdp.drop(columns=['Unnamed: 0'])

    #renaming columns for both csvs
    df_eq = df_eq.rename(columns={"latitude": "latitude_eq", "longitude": "longitude_eq"})
    df_gdp = df_gdp.rename(columns={"latitude": "latitude_gdp", "longitude": "longitude_gdp"})
    
    
    #creating dask dataframes
    ddf_eq = dd.from_pandas(df_eq, npartitions=6).set_index('year')
    ddf_gdp = dd.from_pandas(df_gdp, npartitions=6).set_index('year')

    #merging both data frames: earthquakes and gdps
    ddf_merged = dd.merge(ddf_eq, ddf_gdp, 
                          how="inner",
                          left_index=True, right_index=True).compute()

    #calculation of the distance between the coordinates of each states and each seismic event
    ddf_merged['distance']=ddf_merged.apply(lambda my_data: calc_distance_math(my_data['longitude_gdp'],my_data['latitude_gdp'],my_data['longitude_eq'],my_data['latitude_eq']), axis = 1)

    #filtering just the events that has a distance less than 600 km
    ddf_data = ddf_merged[(ddf_merged.distance <= 600) ]

    return ddf_data



In [ ]:
link_earthquake_gdp_raw = generate_final_data(f_earthquakes_raw,f_gdp_raw)

In [51]:
# Extracting gdp data from csv and managing data for inserted into db

print('Ingesting earthquake from csv file')
f_gdp_raw = pd.read_csv("../data/02.fred_gdp_usa.csv")


print('creating id_earthquake for earthquake data')
f_gdp_raw['id_gdp']=f_gdp_raw.apply(lambda my_data: str(my_data['year'])+str(my_data['latitude'])+str(my_data['longitude']), axis = 1)
f_gdp_raw['id_gdp']=f_gdp_raw.apply(lambda my_data: str(uuid.uuid5(uuid.NAMESPACE_DNS, my_data['id_gdp'])), axis = 1)

print('creating key_earthquake_gdp for earthquake data')
f_gdp_raw['key_gdp_earthquake']=f_gdp_raw.apply(lambda my_data: str(my_data['year'])+str(my_data['latitude'])+str(my_data['longitude']), axis = 1)
f_gdp_raw['key_gdp_earthquake']=f_gdp_raw.apply(lambda my_data: str(uuid.uuid5(uuid.NAMESPACE_DNS, my_data['key_gdp_earthquake'])), axis = 1)

print('Dropping columns for earthquake data')
f_gdp_raw = f_gdp_raw.drop(columns=['Unnamed: 0'])

print('Sorting columns for earthquake data')

column_order = ['id_gdp', 'key_gdp_earthquake', 'year', 'gdp', 'state_code', 'state_name', 'longitude', 'latitude']
f_gdp_raw = f_gdp_raw.reindex(columns=column_order)

print('Saving earthquake data into parquet')
f_gdp_raw.to_parquet('../data/extraction_layer/f_gdp.parquet', engine='fastparquet')

f_gdp_raw

Ingesting earthquake from csv file
creating id_earthquake for earthquake data
creating key_earthquake_gdp for earthquake data
Dropping columns for earthquake data
Sorting columns for earthquake data
Saving earthquake data into parquet


,id_gdp,key_gdp_earthquake,year,gdp,state_code,state_name,longitude,latitude
0,c357db30-7a04-57a2-b16c-b667f737e2ab,c357db30-7a04-57a2-b16c-b667f737e2ab,1997,25810.8,AK,Alaska,-149.680909,64.445961
1,74a01a10-2c2b-58b8-a10b-279737d50eae,74a01a10-2c2b-58b8-a10b-279737d50eae,1998,24227.5,AK,Alaska,-149.680909,64.445961
2,e02a30c1-97b4-5313-81e7-af294bcb66fb,e02a30c1-97b4-5313-81e7-af294bcb66fb,1999,24744.3,AK,Alaska,-149.680909,64.445961
3,48a0aafe-d710-5181-a388-d02c55b3ddd7,48a0aafe-d710-5181-a388-d02c55b3ddd7,2000,26806.6,AK,Alaska,-149.680909,64.445961
4,d4a0bead-c586-5320-a6ad-c039a43fd41d,d4a0bead-c586-5320-a6ad-c039a43fd41d,2001,28494.1,AK,Alaska,-149.680909,64.445961
...,...,...,...,...,...,...,...,...
1245,56c74eac-1d83-5c03-8d23-4163ca51d8e9,56c74eac-1d83-5c03-8d23-4163ca51d8e9,2017,36863.7,WY,Wyoming,-107.568534,43.170026
1246,15e4e337-417e-59af-98b5-5cf63b632685,15e4e337-417e-59af-98b5-5cf63b632685,2018,39032.3,WY,Wyoming,-107.568534,43.170026
1247,9f548c8b-d8fb-5c17-8ee4-5a861c0ab07e,9f548c8b-d8fb-5c17-8ee4-5a861c0ab07e,2019,39600.8,WY,Wyoming,-107.568534,43.170026
1248,c5c61b57-4376-5abe-bf3f-c5a9f7ebf7ca,c5c61b57-4376-5abe-bf3f-c5a9f7ebf7ca,2020,36323.5,WY,Wyoming,-107.568534,43.170026


In [ ]:
# Extracting earthquake data from csv and managing data for inserted into db

print('Ingesting earthquake from csv file')
f_gdp_raw = pd.read_csv("../data/02.fred_gdp_usa.csv")

print('creating id_earthquake for earthquake data')
f_gdp_raw['id_gdp']=f_gdp_raw.apply(lambda my_data: str(my_data['year'])+str(my_data['latitude'])+str(my_data['longitude']), axis = 1)
f_gdp_raw['id_gdp']=f_gdp_raw.apply(lambda my_data: str(uuid.uuid5(uuid.NAMESPACE_DNS, my_data['id_gdp'])), axis = 1)

print('creating key_earthquake_gdp for earthquake data')
f_gdp_raw['key_gdp_earthquake']=f_gdp_raw.apply(lambda my_data: str(my_data['year'])+str(my_data['latitude'])+str(my_data['longitude']), axis = 1)
f_gdp_raw['key_gdp_earthquake']=f_gdp_raw.apply(lambda my_data: str(uuid.uuid5(uuid.NAMESPACE_DNS, my_data['key_gdp_earthquake'])), axis = 1)

print('Dropping columns for earthquake data')
f_earthquakes_raw = f_earthquakes_raw.drop(columns=['Unnamed: 0'])

print('Sorting columns for earthquake data')
column_order = ['id_earthquake', 'key_earthquake_gdp', 'time', 'year', 'latitude', 'longitude',
 'depth', 'mag', 'magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id', 'place',
 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status',
 'locationSource', 'magSource', 'updated']
f_earthquakes_raw = f_earthquakes_raw.reindex(columns=column_order)

print('Renaming columns for earthquake data')
columns_renaming = {"updated":"updated_date"}

f_earthquakes_raw.rename(columns=columns_renaming, inplace=True)

print('Saving earthquake data into parquet')
f_earthquakes_raw.to_parquet('../data/extraction_layer/f_earthquakes.parquet', engine='fastparquet')